In [26]:
import pandas as pd
import numpy as np
import importlib
import ipynb.fs.full.Seismic_Parameters as my
importlib.reload(my)
df = pd.read_csv("../../../data/1st_part.csv")
n=25
from ipynb.fs.full.Seismic_Parameters import time,m_mean,e_mean,a_and_b,Msd,md,low_and_high,ct_and_c,yyy,Msd1
Time = time(df,n)
M_mean = m_mean(df,n)
E_mean = e_mean(df,n)
a,b = a_and_b(df,n)
msd = Msd1(df,n,a,b)
MD = md(df,n)
lowest_mag,highest_mag = low_and_high(df,n)
ct,c = ct_and_c(df,n)
y = yyy(df,n)

In [2]:
#vectors to numpy array to df to csv
#df.to_csv('C:/Users/HP/Downloads/1st_part.csv', index=False)

In [4]:
x_pred_dict={'Time':Time,'M_mean':M_mean,'E_mean':E_mean,'b':b,'a':a,'msd':msd,'MD':MD,'CT':ct,'c':c,'y':y}
new_df = pd.DataFrame(x_pred_dict)
new_df = new_df.iloc[:-1 , :]
new_df

,Time,M_mean,E_mean,b,a,msd,MD,CT,c,y
0,2306,113.9,4.225230e+07,0.444127,0.983267,0.153094,4.086067,646.000000,0.000000,4.3
1,2249,114.1,4.349634e+07,0.448990,0.980977,0.171512,4.115147,682.000000,0.000000,4.6
2,2570,114.4,3.841575e+07,0.466053,0.971782,0.213324,4.214867,1104.000000,0.000000,4.0
3,2354,114.1,4.171167e+07,0.445595,0.974578,0.100399,4.112863,1128.000000,0.000000,4.1
4,2331,113.6,4.156783e+07,0.425835,0.980193,0.134908,3.998188,1138.000000,0.000000,4.4
...,...,...,...,...,...,...,...,...,...,...
132,2165,120.6,5.371605e+07,0.559495,0.958170,0.261169,4.187436,1929.666667,3897.874014,4.6
133,2165,120.8,5.401805e+07,0.568851,0.955369,0.282634,4.220528,1930.000000,3898.303627,5.2
134,2171,121.5,5.590742e+07,0.575559,0.960317,0.343963,4.231506,1973.333333,3984.242005,5.0
135,2194,122.1,5.663489e+07,0.595710,0.960081,0.345654,4.288340,1997.333333,4031.790498,5.0


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
target = new_df['y']
# features = df1.drop(['mag','time','magType'],axis=1)
features = new_df[['Time','M_mean','E_mean','b','a','msd','MD','CT','c']]
target = target.to_numpy()
features = np.array(features)
train_features, test_features, train_target, test_target = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.33, 
                                                                            random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_features[:, :] = sc.fit_transform(train_features[:, :])
test_features[:, :] = sc.transform(test_features[:, :])
train_features.shape[1]

9

In [6]:
network = models.Sequential()

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=128*2, activation='tanh', input_shape=(train_features.shape[1],)))

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=64*2, activation='tanh'))

# Add fully connected layer with no activation function
network.add(layers.Dense(units=1))

In [7]:
# Compile neural network
network.compile(loss='mse', # Mean squared error
                optimizer='RMSprop', # Optimization algorithm
                metrics=['mse','mae']) # Mean squared error

In [8]:
network.fit(train_features, # Features
                      train_target, # Target vector
                      epochs=495, # Number of epochs
                      verbose=1, # No output
                      batch_size=100, # Number of observations per batch
                      validation_data=(test_features, test_target)) # Data for evaluation

Train on 91 samples, validate on 46 samples
Epoch 1/495
91/91 [==============================] - 2s 20ms/sample - loss: 22.3264 - mse: 22.3264 - mae: 4.6956 - val_loss: 18.9474 - val_mse: 18.9474 - val_mae: 4.2837
Epoch 2/495
91/91 [==============================] - 0s 253us/sample - loss: 21.1365 - mse: 21.1365 - mae: 4.5268 - val_loss: 18.8814 - val_mse: 18.8814 - val_mae: 4.2818
Epoch 3/495
91/91 [==============================] - 0s 306us/sample - loss: 20.2488 - mse: 20.2488 - mae: 4.4369 - val_loss: 16.9778 - val_mse: 16.9778 - val_mae: 4.0744
Epoch 4/495
91/91 [==============================] - 0s 317us/sample - loss: 19.0444 - mse: 19.0444 - mae: 4.3130 - val_loss: 16.7241 - val_mse: 16.7241 - val_mae: 4.0290
Epoch 5/495
91/91 [==============================] - 0s 252us/sample - loss: 18.0095 - mse: 18.0095 - mae: 4.1923 - val_loss: 15.0444 - val_mse: 15.0444 - val_mae: 3.8342
Epoch 6/495
91/91 [==============================] - 0s 241us/sample - loss: 17.0173 - mse: 17.0173 - 

Epoch 50/495
91/91 [==============================] - 0s 285us/sample - loss: 0.4342 - mse: 0.4342 - mae: 0.5170 - val_loss: 0.5242 - val_mse: 0.5242 - val_mae: 0.5839
Epoch 51/495
91/91 [==============================] - 0s 266us/sample - loss: 0.4264 - mse: 0.4264 - mae: 0.5221 - val_loss: 0.5300 - val_mse: 0.5300 - val_mae: 0.5788
Epoch 52/495
91/91 [==============================] - 0s 323us/sample - loss: 0.3994 - mse: 0.3994 - mae: 0.4911 - val_loss: 0.4709 - val_mse: 0.4709 - val_mae: 0.5530
Epoch 53/495
91/91 [==============================] - 0s 310us/sample - loss: 0.4019 - mse: 0.4019 - mae: 0.5131 - val_loss: 0.5461 - val_mse: 0.5461 - val_mae: 0.6141
Epoch 54/495
91/91 [==============================] - 0s 296us/sample - loss: 0.4118 - mse: 0.4118 - mae: 0.4922 - val_loss: 0.4929 - val_mse: 0.4929 - val_mae: 0.5509
Epoch 55/495
91/91 [==============================] - 0s 313us/sample - loss: 0.4711 - mse: 0.4711 - mae: 0.5659 - val_loss: 0.7058 - val_mse: 0.7058 - val_mae:

Epoch 99/495
91/91 [==============================] - 0s 307us/sample - loss: 0.3630 - mse: 0.3630 - mae: 0.4998 - val_loss: 0.3830 - val_mse: 0.3830 - val_mae: 0.4917
Epoch 100/495
91/91 [==============================] - 0s 274us/sample - loss: 0.3151 - mse: 0.3151 - mae: 0.4294 - val_loss: 0.4168 - val_mse: 0.4168 - val_mae: 0.4962
Epoch 101/495
91/91 [==============================] - 0s 285us/sample - loss: 0.2810 - mse: 0.2810 - mae: 0.4339 - val_loss: 0.3353 - val_mse: 0.3353 - val_mae: 0.4496
Epoch 102/495
91/91 [==============================] - 0s 348us/sample - loss: 0.2607 - mse: 0.2607 - mae: 0.3835 - val_loss: 0.3802 - val_mse: 0.3802 - val_mae: 0.4676
Epoch 103/495
91/91 [==============================] - 0s 318us/sample - loss: 0.2506 - mse: 0.2506 - mae: 0.4082 - val_loss: 0.3279 - val_mse: 0.3279 - val_mae: 0.4442
Epoch 104/495
91/91 [==============================] - 0s 318us/sample - loss: 0.2481 - mse: 0.2481 - mae: 0.3716 - val_loss: 0.3822 - val_mse: 0.3822 - val

Epoch 148/495
91/91 [==============================] - 0s 322us/sample - loss: 0.2761 - mse: 0.2761 - mae: 0.4263 - val_loss: 0.3299 - val_mse: 0.3299 - val_mae: 0.4614
Epoch 149/495
91/91 [==============================] - 0s 307us/sample - loss: 0.2871 - mse: 0.2871 - mae: 0.4399 - val_loss: 0.4830 - val_mse: 0.4830 - val_mae: 0.5516
Epoch 150/495
91/91 [==============================] - 0s 358us/sample - loss: 0.3073 - mse: 0.3073 - mae: 0.4428 - val_loss: 0.3673 - val_mse: 0.3673 - val_mae: 0.4992
Epoch 151/495
91/91 [==============================] - 0s 306us/sample - loss: 0.3149 - mse: 0.3149 - mae: 0.4628 - val_loss: 0.4647 - val_mse: 0.4647 - val_mae: 0.5444
Epoch 152/495
91/91 [==============================] - 0s 317us/sample - loss: 0.2944 - mse: 0.2944 - mae: 0.4289 - val_loss: 0.3478 - val_mse: 0.3478 - val_mae: 0.4737
Epoch 153/495
91/91 [==============================] - 0s 285us/sample - loss: 0.2727 - mse: 0.2727 - mae: 0.4346 - val_loss: 0.4003 - val_mse: 0.4003 - va

Epoch 197/495
91/91 [==============================] - 0s 275us/sample - loss: 0.3086 - mse: 0.3086 - mae: 0.4511 - val_loss: 0.5200 - val_mse: 0.5200 - val_mae: 0.5746
Epoch 198/495
91/91 [==============================] - 0s 284us/sample - loss: 0.3264 - mse: 0.3264 - mae: 0.4671 - val_loss: 0.3598 - val_mse: 0.3598 - val_mae: 0.4916
Epoch 199/495
91/91 [==============================] - 0s 311us/sample - loss: 0.3135 - mse: 0.3135 - mae: 0.4625 - val_loss: 0.4763 - val_mse: 0.4763 - val_mae: 0.5412
Epoch 200/495
91/91 [==============================] - 0s 317us/sample - loss: 0.2923 - mse: 0.2923 - mae: 0.4338 - val_loss: 0.3492 - val_mse: 0.3492 - val_mae: 0.4760
Epoch 201/495
91/91 [==============================] - 0s 307us/sample - loss: 0.2673 - mse: 0.2673 - mae: 0.4311 - val_loss: 0.4034 - val_mse: 0.4034 - val_mae: 0.5091
Epoch 202/495
91/91 [==============================] - 0s 299us/sample - loss: 0.2396 - mse: 0.2396 - mae: 0.3847 - val_loss: 0.3405 - val_mse: 0.3405 - va

...

Epoch 295/495
91/91 [==============================] - 0s 298us/sample - loss: 0.2162 - mse: 0.2162 - mae: 0.3654 - val_loss: 0.4399 - val_mse: 0.4399 - val_mae: 0.5231
Epoch 296/495
91/91 [==============================] - 0s 351us/sample - loss: 0.2479 - mse: 0.2479 - mae: 0.4105 - val_loss: 0.3293 - val_mse: 0.3293 - val_mae: 0.4707
Epoch 297/495
91/91 [==============================] - 0s 306us/sample - loss: 0.2621 - mse: 0.2621 - mae: 0.4189 - val_loss: 0.4520 - val_mse: 0.4520 - val_mae: 0.5298
Epoch 298/495
91/91 [==============================] - 0s 312us/sample - loss: 0.2597 - mse: 0.2597 - mae: 0.4114 - val_loss: 0.3209 - val_mse: 0.3209 - val_mae: 0.4549
Epoch 299/495
91/91 [==============================] - 0s 296us/sample - loss: 0.2309 - mse: 0.2309 - mae: 0.3999 - val_loss: 0.3794 - val_mse: 0.3794 - val_mae: 0.4860
Epoch 300/495
91/91 [==============================] - 0s 351us/sample - loss: 0.2019 - mse: 0.2019 - mae: 0.3600 - val_loss: 0.2979 - val_mse: 0.2979 - va

Epoch 344/495
91/91 [==============================] - 0s 274us/sample - loss: 0.2393 - mse: 0.2393 - mae: 0.3983 - val_loss: 0.3100 - val_mse: 0.3100 - val_mae: 0.4524
Epoch 345/495
91/91 [==============================] - 0s 307us/sample - loss: 0.2201 - mse: 0.2201 - mae: 0.3855 - val_loss: 0.3805 - val_mse: 0.3805 - val_mae: 0.4828
Epoch 346/495
91/91 [==============================] - 0s 306us/sample - loss: 0.1972 - mse: 0.1972 - mae: 0.3599 - val_loss: 0.2866 - val_mse: 0.2866 - val_mae: 0.4135
Epoch 347/495
91/91 [==============================] - 0s 318us/sample - loss: 0.1752 - mse: 0.1752 - mae: 0.3462 - val_loss: 0.3329 - val_mse: 0.3329 - val_mae: 0.4544
Epoch 348/495
91/91 [==============================] - 0s 296us/sample - loss: 0.1614 - mse: 0.1614 - mae: 0.3188 - val_loss: 0.2819 - val_mse: 0.2819 - val_mae: 0.3933
Epoch 349/495
91/91 [==============================] - 0s 279us/sample - loss: 0.1551 - mse: 0.1551 - mae: 0.3241 - val_loss: 0.3149 - val_mse: 0.3149 - va

Epoch 393/495
91/91 [==============================] - 0s 313us/sample - loss: 0.1643 - mse: 0.1643 - mae: 0.3313 - val_loss: 0.3268 - val_mse: 0.3268 - val_mae: 0.4458
Epoch 394/495
91/91 [==============================] - 0s 274us/sample - loss: 0.1509 - mse: 0.1509 - mae: 0.3128 - val_loss: 0.2672 - val_mse: 0.2672 - val_mae: 0.3823
Epoch 395/495
91/91 [==============================] - 0s 285us/sample - loss: 0.1426 - mse: 0.1426 - mae: 0.3071 - val_loss: 0.3083 - val_mse: 0.3083 - val_mae: 0.4348
Epoch 396/495
91/91 [==============================] - 0s 306us/sample - loss: 0.1392 - mse: 0.1392 - mae: 0.2945 - val_loss: 0.2746 - val_mse: 0.2746 - val_mae: 0.3803
Epoch 397/495
91/91 [==============================] - 0s 295us/sample - loss: 0.1408 - mse: 0.1408 - mae: 0.3055 - val_loss: 0.3085 - val_mse: 0.3085 - val_mae: 0.4404
Epoch 398/495
91/91 [==============================] - 0s 296us/sample - loss: 0.1471 - mse: 0.1471 - mae: 0.2920 - val_loss: 0.3144 - val_mse: 0.3144 - va

Epoch 442/495
91/91 [==============================] - 0s 299us/sample - loss: 0.1333 - mse: 0.1333 - mae: 0.2942 - val_loss: 0.2549 - val_mse: 0.2549 - val_mae: 0.3740
Epoch 443/495
91/91 [==============================] - 0s 275us/sample - loss: 0.1281 - mse: 0.1281 - mae: 0.2861 - val_loss: 0.3000 - val_mse: 0.3000 - val_mae: 0.4244
Epoch 444/495
91/91 [==============================] - 0s 318us/sample - loss: 0.1267 - mse: 0.1267 - mae: 0.2840 - val_loss: 0.2590 - val_mse: 0.2590 - val_mae: 0.3689
Epoch 445/495
91/91 [==============================] - 0s 318us/sample - loss: 0.1281 - mse: 0.1281 - mae: 0.2882 - val_loss: 0.3038 - val_mse: 0.3038 - val_mae: 0.4339
Epoch 446/495
91/91 [==============================] - 0s 307us/sample - loss: 0.1328 - mse: 0.1328 - mae: 0.2841 - val_loss: 0.2840 - val_mse: 0.2840 - val_mae: 0.3809
Epoch 447/495
91/91 [==============================] - 0s 296us/sample - loss: 0.1441 - mse: 0.1441 - mae: 0.3081 - val_loss: 0.3226 - val_mse: 0.3226 - va

Epoch 491/495
91/91 [==============================] - 0s 318us/sample - loss: 0.1394 - mse: 0.1394 - mae: 0.2967 - val_loss: 0.3127 - val_mse: 0.3127 - val_mae: 0.4279
Epoch 492/495
91/91 [==============================] - 0s 309us/sample - loss: 0.1307 - mse: 0.1307 - mae: 0.2932 - val_loss: 0.2458 - val_mse: 0.2458 - val_mae: 0.3738
Epoch 493/495
91/91 [==============================] - 0s 275us/sample - loss: 0.1223 - mse: 0.1223 - mae: 0.2754 - val_loss: 0.2990 - val_mse: 0.2990 - val_mae: 0.4195
Epoch 494/495
91/91 [==============================] - 0s 312us/sample - loss: 0.1188 - mse: 0.1188 - mae: 0.2778 - val_loss: 0.2416 - val_mse: 0.2416 - val_mae: 0.3650
Epoch 495/495
91/91 [==============================] - 0s 336us/sample - loss: 0.1168 - mse: 0.1168 - mae: 0.2678 - val_loss: 0.3020 - val_mse: 0.3020 - val_mae: 0.4245


In [9]:
network.predict(test_features)

array([[4.505841 ],
       [4.5979657],
       [4.5847483],
       [5.2887874],
       [4.402303 ],
       [4.574259 ],
       [4.7269287],
       [4.436658 ],
       [4.279626 ],
       [4.8955407],
       [4.728228 ],
       [4.9222655],
       [4.882103 ],
       [4.6261287],
       [4.128306 ],
       [4.890925 ],
       [4.688026 ],
       [4.7564416],
       [4.573081 ],
       [4.564682 ],
       [4.7745485],
       [4.39351  ],
       [4.6317177],
       [4.4784164],
       [3.9800248],
       [4.1079993],
       [4.7103214],
       [4.449535 ],
       [5.018085 ],
       [4.662338 ],
       [4.937296 ],
       [4.627944 ],
       [4.6411037],
       [4.649838 ],
       [4.5338106],
       [4.2394423],
       [4.705721 ],
       [5.167671 ],
       [4.4171457],
       [4.5422826],
       [5.035649 ],
       [4.7475686],
       [4.440301 ],
       [4.7431917],
       [4.5276346],
       [5.163328 ]], dtype=float32)

In [11]:
test_target


array([4.9, 4.5, 4.6, 4.9, 4. , 4. , 4.1, 4.4, 6. , 4.6, 4.5, 4.4, 4.1,
       3.9, 4.5, 4.3, 4.6, 4.7, 4.8, 4.3, 3.9, 4.4, 4.9, 5. , 4.3, 3.8,
       4.1, 4. , 4.7, 4. , 4. , 5.6, 5. , 4.7, 4.8, 4.6, 4.1, 3.8, 4.3,
       4.5, 4.6, 4.1, 4.4, 4.6, 4.7, 4.9])

In [19]:
network.save('ANN1st.model')
network.save_weights('ANN1st.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ANN1st.model\assets


In [20]:
# network.predict([[2.0,3.0,75.0]])
a1 = network.predict(test_features)

In [21]:
a2 =test_target

In [30]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# real value
TP=0
FP=0
a1=np.round_(a1,decimals = 1)
for i in range(0,len(a1)):
    print(a2[i]," ",a1[i])
errors = mean_squared_error(a2, a1)
print(errors)
print(mean_absolute_error(a2,a1))

4.9   [4.4]
4.5   [4.1]
4.6   [4.3]
4.9   [5.3]
4.0   [4.3]
4.0   [4.2]
4.1   [4.7]
4.4   [4.2]
6.0   [4.3]
4.6   [4.8]
4.5   [4.2]
4.4   [4.8]
4.1   [4.6]
3.9   [4.2]
4.5   [3.6]
4.3   [4.8]
4.6   [4.6]
4.7   [4.7]
4.8   [4.9]
4.3   [4.1]
3.9   [4.2]
4.4   [4.]
4.9   [4.7]
5.0   [4.4]
4.3   [4.3]
3.8   [3.8]
4.1   [4.2]
4.0   [4.]
4.7   [5.1]
4.0   [4.2]
4.0   [4.2]
5.6   [4.4]
5.0   [4.9]
4.7   [4.5]
4.8   [4.3]
4.6   [4.5]
4.1   [4.5]
3.8   [4.9]
4.3   [4.4]
4.5   [4.6]
4.6   [5.3]
4.1   [4.2]
4.4   [4.2]
4.6   [4.8]
4.7   [4.2]
4.9   [5.2]
0.23434781779415975
0.3521739223729009
